In [ ]:
# matplotlib 한글 깨짐 해결
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 이 셀 실행 후 런터임 재실행을 실시하세요

#LightGBM
## Voting 1

#### PCA를 고려하였지만, 성능의 저하가 일어났기에 PCA는 적용하지 않았습니다.


In [ ]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import lightgbm as lgb
from sklearn.decomposition import PCA


import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') # 한글설정

In [ ]:
# 파이썬 지수를 정수로 표현!
pd.options.display.float_format = '{:.5f}'.format

# 코랩 그래프 크게!

%matplotlib inline

plt.rcParams['figure.figsize'] = (25, 25)

In [ ]:

# 한글설정
plt.rc('font', family='NanumBarunGothic')

# 파이썬 지수를 정수로 표현
pd.options.display.float_format = '{:.5f}'.format

# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# 데이터 불러오기
test = pd.read_csv('/content/drive/My Drive/2020 빅콘테스트 예측/lightGBM/2020test.csv', encoding = 'cp949')  ## 자신의 구글 드라이브 경로를 지정하세요!
train = pd.read_csv('/content/drive/My Drive/2020 빅콘테스트 예측/lightGBM/2020train.csv', encoding = 'cp949') ## 자신의 구글 드라이브 경로를 지정하세요!




In [ ]:
print('test', test.columns)
print('train', train.columns)

print(test.shape)
print(train.shape)

#lightGBM 3-Fold Cross Vlaidation을 적용한 함수입니다.

In [ ]:
# MAPE 구해주는 함수
def func_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

## 취급액boxcox > 취급액변환
def original_target(boxcox):
    return (0.2222222 * boxcox +1)**(1/0.2222222)

# 각각의 Valdation set마다 PCA를 적용해 주고, cross validation까지 수행해주는 함수
def PCA_CV(K, train, params, all_feature, pca_feature, target, onehot_feature):
    all_mape = []
    cnt = 1
    np.random.seed(1234)
    kfold = KFold(n_splits=K, shuffle=False)
    for t, v in kfold.split(train):
        # train / val > index 부여
        train_cv = train.iloc[t]
        val_cv = train.iloc[v]

        # train : x, y 나누기
        x_train = train_cv.loc[:, all_feature]
        y_train = train_cv.loc[:, target]
        y_train = y_train.iloc[:, 0]

        # val : x, y 나누기
        x_val = val_cv.loc[:, all_feature]
        y_val = val_cv.loc[:, target]
        y_val = y_val.iloc[:, 0]
        

        if len(pca_feature) != 0 :

            # PCA를 진행할 칼럼만 추출
            x_train_PCA = x_train.loc[:, pca_feature]
            remain_x_train = x_train.drop(pca_feature, axis=1, inplace = False).reset_index(drop = True)
        
            x_val_PCA = x_val.loc[:, pca_feature]
            remain_x_val = x_val.drop(pca_feature, axis=1, inplace = False).reset_index(drop = True)
            
            # PCA를 위한 Z-정규화
            m = np.mean(x_train_PCA)
            s = np.std(x_train_PCA)
            standard_x_train_PCA = (x_train_PCA - m) / s
            standard_x_val_PCA = (x_val_PCA - m) / s

            
            # PCA진행
            # 1. train_set
            pca1 = PCA(n_components=5)    ########################################### 임의로 PCA의 수를 지정해 줄 수 있어요
            pca_x_train = pca1.fit_transform(standard_x_train_PCA)
            pca_x_train_df = pd.DataFrame(data = pca_x_train
                    , columns = ['pca1', 'pca2', 'pca3'])
            # 2. val_set
            pca2 = PCA(n_components=5)    ########################################### 임의로 PCA의 수를 지정해 줄 수 있어요 (위에랑 같은 숫자로 지정해 주어야 해요 ㅠ)
            pca_x_val = pca2.fit_transform(standard_x_val_PCA)
            pca_x_val_df = pd.DataFrame(data = pca_x_val
                    , columns = ['pca1', 'pca2', 'pca3'])
            

            PCA_x_train = pd.concat([remain_x_train, pca_x_train_df],  axis = 1, sort=False)
            PCA_x_val = pd.concat([remain_x_val, pca_x_val_df],  axis = 1, sort=False)

            # OneHot Encoding
            all_data  = PCA_x_train.append(PCA_x_val)
            onehot_all_data = pd.get_dummies(data = all_data, columns=onehot_feature, drop_first = False)

            x_train = onehot_all_data.iloc[:len(PCA_x_train), ]
            x_val = onehot_all_data.iloc[len(PCA_x_train):, ]


        else :
            all_data = x_train.append(x_val)
            onehot_all_data = pd.get_dummies(data = all_data, columns=onehot_feature, drop_first = False)

            x_train = onehot_all_data.iloc[0:len(x_train), ]
            x_val = onehot_all_data.iloc[len(x_train):len(onehot_all_data), ]

        ############################################################# 변수 전처리 완료!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

        # 모델
        model = lgb.LGBMRegressor(learning_rate = params['learning_rate'],
                                  num_iterations = params['num_iterations'],
                                  max_depth = params['max_depth'],
                                  feature_fraction = params['feature_fraction'],
                                  boosting  = 'dart',
                                  eval_metrics = 'mape')
        model.fit(x_train, y_train)

        y_pred = model.predict(x_val)

        if target[0] == '취급액' :
            mape = func_mape(y_val, y_pred)
            print(cnt, '-Fold MAPE : ' ,mape)
            all_mape.append(mape)
            cnt += 1
        else:
            mape = func_mape(original_target(y_val), original_target(y_pred))
            print(cnt, '-Fold MAPE : ' ,mape)
            all_mape.append(mape)
            cnt += 1
    

    print('---------------------------------')
    print('3-Fold Mean MAPE : ', np.mean(all_mape))
    print('모든 mape : ', all_mape)
    print(' ')

    #변수중요도 플랏
    # df = {'feature' : pd.Series(x_train.columns), 'importance' : pd.Series(model.feature_importances_)}
    # importance_df = pd.DataFrame(df)
    # importance_df = importance_df.sort_values(by = ['importance'])
    # importance_df.plot.barh(y = 'importance', x = 'feature')

    return y_pred, y_val, np.mean(all_mape)

In [ ]:
# 실행해서 우리 데이터에 어떤 칼럼이 존재하는지 보세요
# 'Unnamed: 0'은 index번호라서 무조건 뺴면되요~

print('train', train.columns)

In [ ]:
#lightGBM 최종 변수

all_feature = ['판매단가', '노출(분)', 
'월', '일', '시간hour',
'168시간', '휴일', '방송시간', '매진여부',
'결제수단', '기온', '강수량', '풍속', '습도', '적설',
'전운량', '미세먼지', '초미세먼지', '판매단가순위',
'판매단가rank', '중분류순위', '소분류순위',
'방송내_순서', '방송순서', '판매상품종류수', '그룹코드_전체횟수', '그룹코드_대박횟수',
'그룹코드_대박확률','그룹코드_쪽박횟수', '그룹코드_쪽박확률',
'168시간_COS', '168시간_SIN', '24시간_COS', '24시간_SIN', '그룹코드_쪽박횟수', '그룹코드_쪽박확률',
'방송내_상품종류별_점수', '방송내_상품선호도']

In [ ]:
K = 3

all_feature = ['판매단가', '노출(분)', 
'월', '일', '시간hour',
'168시간', '휴일', '방송시간', '매진여부',
'결제수단', '기온', '강수량', '풍속', '습도', '적설',
'전운량', '미세먼지', '초미세먼지', '판매단가순위',
'판매단가rank', '중분류순위', '소분류순위',
'방송내_순서', '방송순서', '판매상품종류수', '그룹코드_전체횟수', '그룹코드_대박횟수',
'그룹코드_대박확률','그룹코드_쪽박횟수', '그룹코드_쪽박확률',
'168시간_COS', '168시간_SIN', '24시간_COS', '24시간_SIN', '그룹코드_쪽박횟수', '그룹코드_쪽박확률',
'방송내_상품종류별_점수', '방송내_상품선호도']
# PCA를 수행하고자 하는 변수를 입력하세요
pca_feature = []

# Target 변수를 입력하세요 '취급액'으로 넣어도 됩니다.
target = ['취급액boxcox']

# all_feature에 입력했던 범주형 변수를 입력하세요
onehot_feature = ['휴일', '매진여부', '결제수단']
# 파라매터를 입력하세요
params = {'learning_rate' : 0.1, 'num_iterations' : 1200, 'max_depth' : 8, 'feature_fraction' : 0.7}

In [ ]:
# Grid search 없이 cross validation만 수행

y_pred, y_real, temp = PCA_CV(K, train, params, all_feature, pca_feature, target, onehot_feature)


In [ ]:
# 실제 Target과 예측 Target의 그래프 추이
plt.rcParams['figure.figsize'] = (20, 5)

data = {
    'real' : y_real,
    'pred' : y_pred
}


data
df = pd.DataFrame(data)

df.plot.line(linewidth = 0.5)

#lightGBM Grid search 구현입니다

##최적의 하이퍼 파라메터
learning_rate': 0.1
num_iterations': 1200
max_depth': 12 
feature_fraction: 0.8}



In [ ]:
# Gridsearch 구현




np.random.seed(1234)
min = 99999
for learning_rate in [0.1, 0.2]:  # learning_rate
    for num_iterations in [1200, 1400]: #num_iterations
        for max_depth in [12, 10] : # max_depth
            for feature_fraction in [0.7, 0.8]:
                params = {'learning_rate' : learning_rate, 'num_iterations' : num_iterations, 'max_depth' : max_depth, 'feature_fraction' : feature_fraction}
                y_pred, y_real, temp = PCA_CV(K, train, params, all_feature, pca_feature, target, onehot_feature)
                if min > temp :
                    min = temp
                    best_param = params
print()
print()
print()
print('가장 낮은 MAPE : ', min) # 가장 낮은 MAPE를 출력합니다
print('Best parametor :', best_param) # MAPE가 가장 낮을때의 파라매터를 출력합니다


In [ ]:
print(min)
print(best_param)

# 1
# 36.0792166370992
# {'learning_rate': 0.1, 'num_iterations': 1200, 'max_depth': 12, 'feature_fraction': 0.8}

In [ ]:
kfold = KFold(n_splits=3, shuffle=False)

for a, b in kfold.split(train):
    print('train : ', a)
    print('test :', b)

In [ ]:
### 최종예측!

## 취급액boxcox > 취급액변환
def original_target(boxcox):
    return (0.2222222 * boxcox +1)**(1/0.2222222)

# MAPE 구해주는 함수
def func_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

### Final Predict
def final_predict(train, test, params, all_feature, pca_feature, target, onehot_feature):
    # train : x, y 나누기
    np.random.seed(1234)
    x_train = train.loc[:, all_feature]
    y_train = train.loc[:, target]
    y_train = y_train.iloc[:, 0]

    # test : x, y 나누기
    x_test = test.loc[:, all_feature]
    y_test = test.loc[:, target]
    y_test = y_test.iloc[:, 0]

    if len(pca_feature) != 0 :
        # PCA를 진행할 칼럼만 추출
        x_train_PCA = x_train.loc[:, pca_feature]
        remain_x_train = x_train.drop(pca_feature, axis=1, inplace = False).reset_index(drop = True)
        
        x_val_PCA = x_test.loc[:, pca_feature]
        remain_x_test = x_test.drop(pca_feature, axis=1, inplace = False).reset_index(drop = True)
            
            # PCA를 위한 Z-정규화
        m = np.mean(x_train_PCA)
        s = np.std(x_train_PCA)
        standard_x_train_PCA = (x_train_PCA - m) / s
        standard_x_test_PCA = (x_test_PCA - m) / s

            
        # PCA진행
        # 1. train_set
        pca1 = PCA(n_components=5)    ########################################### 임의로 PCA의 수를 지정해 줄 수 있어요
        pca_x_train = pca1.fit_transform(standard_x_train_PCA)
        pca_x_train_df = pd.DataFrame(data = pca_x_train
                , columns = ['pca1', 'pca2', 'pca3', 'pca4', 'pca5'])
        # 2. val_set
        pca2 = PCA(n_components=5)    ########################################### 임의로 PCA의 수를 지정해 줄 수 있어요 (위에랑 같은 숫자로 지정해 주어야 해요 ㅠ)
        pca_x_test = pca2.fit_transform(standard_x_test_PCA)
        pca_x_test_df = pd.DataFrame(data = pca_x_val
                , columns = ['pca1', 'pca2', 'pca3', 'pca4', 'pca5'])
            

        PCA_x_train = pd.concat([remain_x_train, pca_x_train_df],  axis = 1, sort=False)
        PCA_x_val = pd.concat([remain_x_val, pca_x_val_df],  axis = 1, sort=False)

        # OneHot Encoding
        all_data  = PCA_x_train.append(PCA_x_test)
        onehot_all_data = pd.get_dummies(data = all_data, columns=onehot_feature, drop_first = False)

        x_train = onehot_all_data.iloc[:len(PCA_x_train), ]
        x_test = onehot_all_data.iloc[len(PCA_x_train):, ]        

    else:
        all_data = x_train.append(x_test)
        onehot_all_data = pd.get_dummies(data = all_data, columns=onehot_feature, drop_first = False)

        x_train = onehot_all_data.iloc[0:len(x_train), ]
        x_test = onehot_all_data.iloc[len(x_train):len(onehot_all_data), ]        

    ## 예측시작
    np.random.seed(1234)
    model = lgb.LGBMRegressor(learning_rate = params['learning_rate'],
                              num_iterations = params['num_iterations'],
                              max_depth = params['max_depth'],
                              feature_fraction = params['feature_fraction'],
                              boosting  = 'dart',
                              eval_metrics = 'mape')
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    if target[0] == '취급액':
        y_pred = y_pred
        y_test = np.array(y_test)
        mape = func_mape(y_test, y_pred)
        print('Final MAPE : ' ,mape)


    else:
        y_pred = original_target(y_pred)
        y_test = original_target(y_test)
        mape = func_mape(y_test, y_pred)
        print('Final MAPE : ' ,mape)
        
    
    # 변수중요도 플랏
    df = {'feature' : pd.Series(x_train.columns), 'importance' : pd.Series(model.feature_importances_)}
    importance_df = pd.DataFrame(df)
    importance_df = importance_df.sort_values(by = ['importance'])
    importance_df.plot.barh(y = 'importance', x = 'feature')

    return y_pred, y_test, mape

In [ ]:
# 실제 Target과 예측 Target의 그래프 추이
plt.rcParams['figure.figsize'] = (200, 40)

data = {
    'real' : y_real,
    'pred' : y_pred
}


data
df = pd.DataFrame(data)

df.plot.line(linewidth = 1)

In [ ]:
K = 3

all_feature = ['판매단가', '노출(분)', 
'월', '일', '시간hour',
'168시간', '휴일', '방송시간', '매진여부',
'결제수단', '기온', '강수량', '풍속', '습도', '적설',
'전운량', '미세먼지', '초미세먼지', '판매단가순위',
'판매단가rank', '중분류순위', '소분류순위',
'방송내_순서', '방송순서', '판매상품종류수', '그룹코드_전체횟수', '그룹코드_대박횟수',
'그룹코드_대박확률','그룹코드_쪽박횟수', '그룹코드_쪽박확률',
'168시간_COS', '168시간_SIN', '24시간_COS', '24시간_SIN', '그룹코드_쪽박횟수', '그룹코드_쪽박확률',
'방송내_상품종류별_점수', '방송내_상품선호도']
# PCA를 수행하고자 하는 변수를 입력하세요
pca_feature = []

# Target 변수를 입력하세요 '취급액'으로 넣어도 됩니다.
target = ['취급액boxcox']

# all_feature에 입력했던 범주형 변수를 입력하세요
onehot_feature = ['휴일', '매진여부', '결제수단']
# 파라매터를 입력하세요
params = {'learning_rate' : 0.1, 'num_iterations' : 1200, 'max_depth' : 8, 'feature_fraction' : 0.7}

In [ ]:
# test set : 2019.06 예측

y_pred, y_real, mape = final_predict(train, test, params, all_feature, pca_feature, target, onehot_feature)


# DNN과 결합하기 위해 csv 파일로 저장
## Voting 결과는 '2020_DNN_lightGBM_2020년06월_예측.csv' 파일에서 확인해 주시길 바랍니다.

In [ ]:
data = {
    'real' : y_real,
    'pred' : y_pred,
    'minus' : abs((y_real - y_pred)/y_real) * 100
}



df = pd.DataFrame(data)

newdf = pd.concat([test.loc[:, ['방송일시', '상품명', '상품군']], df], axis = 1, sort=False)
newdf.sort_values(['방송일시'])

newdf.to_csv("/content/drive/My Drive/2020 빅콘테스트 예측/lightGBM/개별적MAPE/2020_DNN_lightGBM_2020년06월_예측.csv", encoding='ms949')
